# Bagian 1: Pra-pemrosesan Teks (Bobot: 20%)

Sebelum data diolah, data harus dibersihkan agar model dapat bekerja maksimal. Teknik preprocessing yang digunakan:
1. Case Folding: Mengubah huruf menjadi kecil semua
2. Tokenisasi: Memecah kalimat menjadi kata-kata
3. Stopword Removal: Menghapus kata umum yang tidak bermakna
4. Cleaning: Menghapus tanda baca, angka, atau karakter aneh menggunakan Regex
5. Stemming: Mengubah kata ke bentuk dasarnya

In [ ]:
# Instalasi library tambahan
!pip install sastrawi -q

In [ ]:
# Import library
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Download resource NLTK
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    print(f"Error downloading NLTK resources: {e}")

In [ ]:
# Koneksi ke Google Drive untuk mengakses dataset
print("Menghubungkan ke Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# Atur path dataset dari Google Drive Anda
# Harap sesuaikan path ini dengan lokasi dataset Anda di Google Drive
dataset_path = '/content/drive/MyDrive/dataset/hoasa_absa-airy'

# Cek apakah dataset tersedia di Drive
if os.path.exists(dataset_path):
    print(f"Dataset ditemukan di: {dataset_path}")
    print("File-file yang tersedia dalam dataset:")
    for file in os.listdir(dataset_path):
        print(f"- {file}")
else:
    print(f"Dataset tidak ditemukan di {dataset_path}")
    # Jika dataset tidak ditemukan di Drive, kita akan menggunakan dataset lokal
    dataset_path = '/home/dalemasan/Documents/Project Python/NLP/dataset/hoasa_absa-airy'
    if os.path.exists(dataset_path):
        print(f"Dataset lokal ditemukan di: {dataset_path}")
        print("File-file yang tersedia dalam dataset:")
        for file in os.listdir(dataset_path):
            print(f"- {file}")
    else:
        print(f"Dataset tidak ditemukan di manapun")
        # Jika tetap tidak ada, kita harus keluar
        raise FileNotFoundError("Dataset tidak ditemukan di Drive maupun lokal")

In [ ]:
# Membaca dataset asli dari hoasa_absa-airy dari Google Drive atau lokal
# Coba baca file train_preprocess.csv terlebih dahulu

# Cari file CSV untuk digunakan
csv_files = glob.glob(os.path.join(dataset_path, '*train*.csv'))
if csv_files:
    # Gunakan file pertama yang mengandung 'train' dalam nama
    train_file = csv_files[0]
else:
    # Jika tidak ada file train, cari validasi atau test
    csv_files = glob.glob(os.path.join(dataset_path, '*.csv'))
    train_file = None
    for file in csv_files:
        if 'train' in file:
            train_file = file
            break
    
    if not train_file:
        for file in csv_files:
            if 'valid' in file:
                train_file = file
                break
    
    if not train_file:
        for file in csv_files:
            if 'test' in file:
                train_file = file
                break

# Baca file CSV
df = pd.read_csv(train_file)

print(f"File CSV dibaca: {train_file}")
print(f"Shape dataset: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\n5 baris pertama:")
print(df.head())

# Cek struktur data dan tentukan kolom yang benar
if 'text' in df.columns and 'label' in df.columns:
    # Sudah dalam format yang benar
    print("\nKolom 'text' dan 'label' ditemukan")
elif len(df.columns) >= 2:
    # Gunakan kolom pertama sebagai text dan kedua sebagai label
    df.columns = ['text', 'label'] + df.columns[2:].tolist()
    print("\nKolom diubah menjadi 'text' dan 'label'")
else:
    # Coba tampilkan struktur data untuk analisis
    print(f"Struktur data tidak sesuai. Kolom yang ada: {df.columns.tolist()}")
    
# Ambil minimal 100 data dari dataset asli, atau semua jika kurang dari 100
if len(df) >= 100:
    df = df.head(100)  # Ambil 100 baris pertama
    print(f"\nMenggunakan 100 baris pertama dari dataset")
else:
    print(f"\nDataset hanya memiliki {len(df)} baris (kurang dari 100), menggunakan semua data")

print(f"Dataset final loaded with shape: {df.shape}")
print(df.head(10))

In [ ]:
print("Data sebelum preprocessing (5 baris pertama dari dataset asli):")
df_before = df[['text', 'label']].head()
display(df_before)

In [ ]:
# Inisialisasi stemmer
try:
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
except Exception as e:
    print(f"Error inisialisasi stemmer: {e}")
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

def preprocess_text(text):
    if pd.isna(text):
        return ""
    
    # Cleaning: Hapus karakter selain huruf dan spasi
    text = re.sub(r'[^a-zA-Z\s]', ' ', str(text))
    
    # Case Folding: ubah ke lowercase
    text = text.lower()
    
    # Tokenisasi
    try:
        tokens = word_tokenize(text)
    except Exception:
        # Fallback jika tokenisasi gagal
        tokens = text.split()
    
    # Stopword Removal
    try:
        stop_words = set(stopwords.words('indonesian'))
        tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    except Exception:
        # Jika stopwords tidak tersedia, lewati proses ini
        tokens = [token for token in tokens if len(token) > 2]
    
    # Stemming
    try:
        tokens = [stemmer.stem(token) for token in tokens]
    except Exception:
        # Jika stemming gagal, gunakan token asli
        pass
    
    # Gabung kembali menjadi string
    return ' '.join(tokens)

# Terapkan preprocessing ke kolom text
df['clean_text'] = df['text'].apply(preprocess_text)

print("Proses preprocessing selesai")

In [ ]:
print("Data setelah preprocessing (5 baris pertama):")
df_after = df[['text', 'clean_text', 'label']].head()
display(df_after)

In [ ]:
print("""
TUGAS LAPORAN: Menjelaskan mengapa preprocessing penting dilakukan

Tahap preprocessing penting dilakukan sebelum masuk ke tahap machine learning karena:

1. Case Folding: Menghindari perbedaan antara kata yang sama dengan kapitalisasi berbeda 
   (contoh: 'Hotel' vs 'hotel'), sehingga model tidak menganggapnya sebagai kata berbeda

2. Tokenisasi: Memungkinkan komputer memproses teks sebagai unit-unit kecil (kata-kata) 
   daripada string panjang, memudahkan analisis dan pemrosesan

3. Stopword Removal: Mengurangi noise dari kata-kata umum yang tidak memberi informasi
   signifikan (seperti 'yang', 'dan', 'di'), sehingga model fokus pada kata-kata penting

4. Cleaning: Membersihkan karakter aneh, tanda baca, atau angka yang bisa mengganggu
   proses machine learning, membuat representasi teks lebih konsisten

5. Stemming: Mengurangi variasi kata ke bentuk dasarnya (contoh: 'membangun' -> 'bangun'),
   mengurangi dimensi fitur dan membuat model lebih efisien

Dengan preprocessing yang baik, data menjadi lebih bersih, terstruktur, dan siap diproses
oleh algoritma machine learning, yang akan meningkatkan akurasi dan kinerja model.
""")